Rather than bootstrapping for eps, use MC methods

In [7]:
import gym
import numpy as np
import pandas as pd

In [27]:
print('Num actions: %i'%env.nA)

Num actions: 6


In [2]:
import Box2D
env = gym.make('Blackjack-v0')#.env
rand_pol = np.random.randint(0,6,1000)
obs= env.reset()
#env.render()

Blackjack rules and scoring:
    - first to make 21
    - Reward for stick:
        - +1 if sum of cards > sum of dealer cards 0 if sum of cards = sum of dealer cards -1 if sum of cards < sum of dealer cards
        - Reward for twist:
        - -1 if sum of cards > 21 (and terminate)
        - 0 otherwise

In [3]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Tuple(Discrete(32), Discrete(11), Discrete(2))


In [10]:
def policy(score):
    if score <=12:
        return 1
    elif score >=20:
        return 0
    else:
        r = np.random.randint(1,10)
        if r<=5:
            return 0
        else:
            return 1

def every_visit_mc(env, n_eps=1000):
    all_visits={}
    count_visits={}
    returns ={}
    
    #generate episode 
    for ep in range(n_eps):
        list_s=[]
        
        ep_visits={}
        ep_counts = {}
        ep_returns = {}
        rew=0.0
        
        observation =env.reset()
        action=policy(observation[0])
        
        list_s.append([observation[0], observation[1], action, rew])
        
        for t in range(100):
            
            score, dealer_score, ace = observation
            action = policy(score)
            observation, rew, done, info = env.step(action)
            list_s.append([observation[0], observation[1], action, rew])
            
            if done:
                break

        for (state, d_state, action, rew) in list_s:
            if not (state,action) in ep_visits.keys():
                if (state==20 or state==21) and rew==0.0:
                    continue
                else:
                    ep_visits[(state, d_state, action)]=float(rew)
                    ep_counts[(state, d_state, action)]=1.0
                    all_visits[(state, d_state, action)]=float(rew)
                    count_visits[(state, d_state, action)]=1.0
            else:
                ep_visits[(state, d_state, action)]+=float(rew)
                ep_counts[(state, d_state, action)]+=1.0
                count_visits[(state, d_state, action)]+=1.0
                all_visits[(state, d_state, action)]+=float(rew)
       
        for ((state, d_state, action),rew) in ep_visits.items():
            ep_returns[(state, d_state, action)]=ep_visits[(state, d_state, action)]/ep_counts[(state, d_state,action)]
        #print('Returns for this episode: %s \n'%ep_returns)

    #Take the average across all episodes 
    for ((state, d_state, action), rew) in all_visits.items():
        returns[(state, state, action)]=all_visits[(state, d_state, action)]/count_visits[(state, d_state, action)]
    #print('Final avg returns: %s \n'%returns)
        
    return returns
    
dic = every_visit_mc(env)

print(pd.DataFrame.from_dict(sorted(dic.items(), key= lambda x:x)))

              0    1
0     (4, 4, 1)  0.0
1     (5, 5, 1)  0.0
2     (6, 6, 1)  0.0
3     (7, 7, 1)  0.0
4     (8, 8, 1)  0.0
5     (9, 9, 1)  0.0
6   (10, 10, 1)  0.0
7   (11, 11, 1)  0.0
8   (12, 12, 1)  0.0
9   (13, 13, 0) -1.0
10  (13, 13, 1)  0.0
11  (14, 14, 0)  1.0
12  (14, 14, 1)  0.0
13  (15, 15, 0) -1.0
14  (15, 15, 1)  0.0
15  (16, 16, 0) -1.0
16  (16, 16, 1)  0.0
17  (17, 17, 0) -1.0
18  (17, 17, 1)  0.0
19  (18, 18, 0)  0.0
20  (18, 18, 1)  0.0
21  (19, 19, 0)  0.0
22  (19, 19, 1)  0.0
23  (20, 20, 0)  1.0
24  (21, 21, 0)  1.0
25  (22, 22, 1) -1.0
26  (23, 23, 1) -1.0
27  (24, 24, 1) -1.0
28  (25, 25, 1) -1.0
29  (26, 26, 1) -1.0
30  (27, 27, 1) -1.0
31  (28, 28, 1) -1.0
32  (29, 29, 1) -1.0


Becuase we cannot subtract cards in blackjack, every-visit MC would produce the same results as first-visit